In [1]:
import datetime as dt
import json, os
import pandas as pd
from matplotlib import pyplot as plt
from collections import defaultdict
import numpy as np
from itertools import combinations
from aquabyte.accuracy_metrics import AccuracyMetricsGenerator
from aquabyte.data_access_utils import S3AccessUtils, RDSAccessUtils
from aquabyte.visualize import Visualizer, _normalize_world_keypoints
from aquabyte.optics import euclidean_distance, pixel2world, depth_from_disp, convert_to_world_point
from aquabyte.data_loader import 
import random
import torch
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from sklearn.model_selection import train_test_split
from copy import copy

pd.set_option('display.max_rows', 500)

<h1> Get all IMR data </h1>

In [8]:
rds_access_utils = RDSAccessUtils(json.load(open(os.environ['PROD_SQL_CREDENTIALS'])))
query = """
    select * from keypoint_annotations where pen_id=61 and captured_at >= '2019-09-13' and captured_at <= '2019-09-20';
"""
df = rds_access_utils.extract_from_database(query)

In [11]:
rds_access_utils = RDSAccessUtils(json.load(open(os.environ['PROD_SQL_CREDENTIALS'])))
query = """
    select * from keypoint_annotations 
    where pen_id=61 and captured_at >= '2019-09-13' and captured_at <= '2019-09-20'
    and keypoints is not null
    order by captured_at
    limit 10;
"""
df = rds_access_utils.extract_from_database(query)

In [13]:
for idx, row in df.iterrows():
    print(row.keypoints['leftCrop'])
    print(row.keypoints['rightCrop'])

[{'xCrop': 3545, 'yCrop': 574, 'xFrame': 4056, 'yFrame': 1816, 'keypointType': 'UPPER_LIP'}, {'xCrop': 3507, 'yCrop': 593, 'xFrame': 4018, 'yFrame': 1835, 'keypointType': 'EYE'}, {'xCrop': 2726, 'yCrop': 198, 'xFrame': 3237, 'yFrame': 1440, 'keypointType': 'DORSAL_FIN'}, {'xCrop': 1416, 'yCrop': 289, 'xFrame': 1927, 'yFrame': 1531, 'keypointType': 'ADIPOSE_FIN'}, {'xCrop': 935, 'yCrop': 344, 'xFrame': 1446, 'yFrame': 1586, 'keypointType': 'UPPER_PRECAUDAL_PIT'}, {'xCrop': 585, 'yCrop': 483, 'xFrame': 1096, 'yFrame': 1725, 'keypointType': 'HYPURAL_PLATE'}, {'xCrop': 318, 'yCrop': 486, 'xFrame': 829, 'yFrame': 1728, 'keypointType': 'TAIL_NOTCH'}, {'xCrop': 937, 'yCrop': 643, 'xFrame': 1448, 'yFrame': 1885, 'keypointType': 'LOWER_PRECAUDAL_PIT'}, {'xCrop': 1627, 'yCrop': 841, 'xFrame': 2138, 'yFrame': 2083, 'keypointType': 'ANAL_FIN'}, {'xCrop': 2383, 'yCrop': 954, 'xFrame': 2894, 'yFrame': 2196, 'keypointType': 'PELVIC_FIN'}, {'xCrop': 3240, 'yCrop': 789, 'xFrame': 3751, 'yFrame': 2031, 